In [1]:
import random
import graph

g = graph.Graph()
g.loadGraph("./out/saved_graph_races.pkl")

# g.populateNodes()

# g.saveGraph("./out/saved_graph_races.pkl")

In [12]:
tract_nodes = []
for nid, node in g.nodes.items():
    if(node.population == 0):
        node.population =  {"POP": 0,
            "LATPOP_D": 0,
            "WHIPOP_D": 0,
            "BLAPOP_D": 0,
            "AMIPOP_D": 0,
            "ASIPOP_D": 0,
            "HPIPOP_D": 0,
            "OTHPOP_D": 0,
            "MMRPOP_D": 0,
          }
    

id: (283, 268) , @(34.06830177678373,-118.21232776360196), report_cnt = 0
id: (283, 268) , @(34.06830177678373,-118.21232776360196), report_cnt = 0
{'POP': 194, 'LATPOP_D': 119, 'WHIPOP_D': 2, 'BLAPOP_D': 0, 'AMIPOP_D': 0, 'ASIPOP_D': 69, 'HPIPOP_D': 0, 'OTHPOP_D': 0, 'MMRPOP_D': 0}
{'POP': 194, 'LATPOP_D': 119, 'WHIPOP_D': 2, 'BLAPOP_D': 0, 'AMIPOP_D': 0, 'ASIPOP_D': 69, 'HPIPOP_D': 0, 'OTHPOP_D': 0, 'MMRPOP_D': 0}
True


In [19]:
import patrols
import plotter


sim_by_race = {}
races = ["POP", "LATPOP_D", "WHIPOP_D", "BLAPOP_D", "AMIPOP_D", "ASIPOP_D", "HPIPOP_D", "OTHPOP_D", "MMRPOP_D"]


for race_id in races:
    min_pop = min([node.population[race_id] for nid, node in g.nodes.items()])
    max_pop = max([node.population[race_id] for nid, node in g.nodes.items()])
    
    # p = (cur - min) / (max - min)
    
    arrest_rate = {node_id:((n.population[race_id] - min_pop) / (max_pop - min_pop)) for node_id, n in g.nodes.items()}
    
    all_nodes = {}

    for pID in range(1,22):
        while(True):
            start = random.choice(list(g.precincts[pID].values()))
            try:
                path, arrests = patrols.random_walk_precinct(50000, g, start, arrest_rate)
            except:
                continue
            else:
                all_nodes.update(arrests)
                print(f"Race: {race_id}, precinct: {pID}", end="\r")
                break
            
    sim_by_race[race_id] = all_nodes

In [21]:
import stats

for rID, sim in sim_by_race.items():
    if rID == "HPIPOP_D":
        print(f"{rID}: mean = {stats.sim_mean(sim)}  --  stddev = {stats.sim_stddev(sim)}")
        plotter.basic_plot(sim, f"./out/graphics/{rID}.png")

HPIPOP_D: mean = 0.04773585919174918  --  stddev = 1.2620621016609899


In [18]:

real_arrest_counts = {ids:len(nodes.pd_reports) for ids, nodes in g.nodes.items()}
plotter.basic_plot(real_arrest_counts, "./out/graphics/arrest_count")